In [1]:
import os
import sys

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)


In [2]:
# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from resnet18 import Resnet18
from data_preprocessing import *


2023-07-03 16:11:28.380598: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 16:11:28.415337: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 16:11:28.919852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
batch_size = 128
epoch = 5

In [4]:

# load data
default_training_dataset_path = "../../data/cifar_data/5_chunks/chunk_2.pickle"
default_testing_dataset_path = "../../data/cifar_data/test_set.pickle"


# train_ds, data_size = preprocess_dataset(training_dataset_path, training = True)
# test_ds, _ = preprocess_dataset(testing_dataset_path, training = False)
train_ds, data_size = preprocess_dataset(default_training_dataset_path, batch_size = 128, training = True)
test_ds, _ = preprocess_dataset(default_testing_dataset_path, training = False)


2023-07-03 16:11:30.627726: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22301 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:19:00.0, compute capability: 8.6
2023-07-03 16:11:30.628259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22301 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:8d:00.0, compute capability: 8.6


In [5]:

# define model
model = Resnet18(input_features = (32, 32, 3), output_features = 10, lr=1e-1, decay_steps=int(epoch * data_size / batch_size))
# define framework
tensorflow_framework = TensorflowFramework(model = model, epoch= 200, data_size= data_size, train_ds= train_ds, test_ds= test_ds, regularization='l2', delta_time= 10000, qod= 0.45)



In [6]:
# fit to be able to load model
for images, labels in tensorflow_framework.train_ds:
    train_acc, train_loss= tensorflow_framework.fit(images, labels)
    break


2023-07-03 16:11:31.680122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [4166,32,32,3]
	 [[{{node Placeholder/_0}}]]
2023-07-03 16:11:31.680381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [4166,10]
	 [[{{node Placeholder/_1}}]]
2023-07-03 16:11:34.944001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-03 16:11:35.974058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2

In [7]:
# training_process/cifar_dataset/server/weights/global_weights/d2771e5c-4cd7-470e-a376-aa67686a0ebc_v147.pkl

In [11]:
n = 220
for i in range(20):
    version = n + i
    file_name = f"./weights/global_weights/99ac459f-f602-48ac-b56a-e180beb6839f_v{version}.pkl"

    with open(file_name, "rb") as f:
        import pickle
        weights = pickle.load(f)

    tensorflow_framework.set_weights(weights)

    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("Global Version {}: Test Acc {:.2f}  Test Loss {}".format(version, test_acc  * 100, test_loss) )
    

Global Version 220: Test Acc 88.63  Test Loss 0.6709173917770386
Global Version 221: Test Acc 91.03  Test Loss 0.4979305565357208
Global Version 222: Test Acc 90.70  Test Loss 0.549485981464386
Global Version 223: Test Acc 90.44  Test Loss 0.5539805889129639
Global Version 224: Test Acc 90.13  Test Loss 0.5977513194084167
Global Version 225: Test Acc 87.84  Test Loss 0.7456607222557068
Global Version 226: Test Acc 88.88  Test Loss 0.6804056763648987
Global Version 227: Test Acc 91.05  Test Loss 0.544531524181366
Global Version 228: Test Acc 89.52  Test Loss 0.6665125489234924
Global Version 229: Test Acc 87.38  Test Loss 0.8338127732276917
Global Version 230: Test Acc 91.99  Test Loss 0.4581722319126129
Global Version 231: Test Acc 90.82  Test Loss 0.5650597810745239
Global Version 232: Test Acc 89.96  Test Loss 0.6706516146659851
Global Version 233: Test Acc 89.11  Test Loss 0.6973994970321655
Global Version 234: Test Acc 88.19  Test Loss 0.7642741203308105
Global Version 235: Test Ac